In [25]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
sns.set(style='white',palette='muted',color_codes=True)
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import plotly.express as px 

In [26]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression,LogisticRegression 
from sklearn.metrics import silhouette_score,accuracy_score,r2_score,confusion_matrix,mean_squared_error 
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OrdinalEncoder,OneHotEncoder,LabelEncoder 
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor 
from sklearn.cluster import KMeans 
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [27]:
data = pd.read_csv(r'c:\Github\Fullstack-Data-Analyst\Learning\the_data\data-lab-3-insurance.csv')

In [28]:
data.duplicated().sum()
data.drop_duplicates(subset=None,keep='first',inplace=True,ignore_index=False)
data.duplicated().sum()

0

In [29]:
mis_val = data.isna().sum() * 100/len(data)
mis_val.sort_values(ascending=False)



age         0.0
sex         0.0
bmi         0.0
children    0.0
smoker      0.0
region      0.0
charges     0.0
dtype: float64

In [30]:
le = LabelEncoder()

data.sex = le.fit_transform(data.sex)
data.smoker = le.fit_transform(data.smoker)
data.region = le.fit_transform(data.region)

In [31]:
data.head(3)

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.90,0,1,3,16884.9240
1,18,1,33.77,1,0,2,1725.5523
2,28,1,33.00,3,0,2,4449.4620


In [32]:
scaler = StandardScaler()
x = data.drop(columns=['charges'],axis=1)
y = data.charges 


x = scaler.fit_transform(x)
y = scaler.fit_transform(np.array(y).reshape(-1,1))


In [33]:
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42,test_size=0.3)

In [34]:
linreg = LinearRegression()
linreg.fit(x_train,y_train)

y_pred = linreg.predict(x_test)

mse_scr = mean_squared_error(y_test,y_pred)

mod_perf = pd.DataFrame([
     
        'LinearRegression', mse_scr
     
]).transpose()
mod_perf.columns = ['Method', 'mse test score']
mod_perf

,Method,mse test score
0,LinearRegression,0.265677


In [35]:
ranfor = RandomForestRegressor(n_estimators=100,random_state=42,n_jobs=-1,criterion='squared_error')

ranfor.fit(x_train,y_train)

y_pred = ranfor.predict(x_test)

scr_r2 = r2_score(y_test,y_pred)
ran_perf = pd.DataFrame([
    'RandomForestRegressor', scr_r2
]).transpose()

ran_perf.columns = ['Method', 'Ran Forest MSE score']
ran_perf

C:\Users\amunv\AppData\Local\Temp\ipykernel_19860\620228957.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  ranfor.fit(x_train,y_train)


,Method,Ran Forest MSE score
0,RandomForestRegressor,0.862775
